In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Display the full content of each column


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Display the full content of each column
import re
from tqdm import tqdm
tqdm.pandas()  # This adds the tqdm progress bar to pandas operations
from openai import OpenAI
client = OpenAI()
from dotenv import load_dotenv
import json
ret = load_dotenv()
import ast

In [3]:
df1 = pd.read_csv("../data/benz_packge.csv")

In [4]:
df2 = pd.read_csv("../data/benz_rebate.csv")

In [5]:
df1.head(2)

,Package,Rate Model,Bank,MSRP,Invoice,Formula,Buying,Default Rebates,Nondefault Rebates,Rate Style,MF 12,RP 12,Live 12,MF 24,RP 24,Live 24,MF 27,RP 27,Live 27,MF 30,RP 30,Live 30,MF 33,RP 33,Live 33,MF 36,RP 36,Live 36,MF 39,RP 39,Live 39,MF 42,RP 42,Live 42,MF 48,RP 48,Live 48,Package Id,Bank Id
0,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,110895.0,104798.0,0.945020,-3000.0,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64763,2
1,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,116005.0,109601.0,0.944795,-3000.0,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64764,2


In [6]:
df1.shape

(1428, 39)

In [7]:
df2.head(2)

,MatrixPackage,Year,Bank,Code,MatrixRateModel,MatrixRateStyle,EndDate,MatirxRebatesSum,AISRebatesSum,Rebate1,RebateDefault1,MatrixValue1,AISValue1,Rebate2,RebateDefault2,MatrixValue2,AISValue2,Rebate3,RebateDefault3,MatrixValue3,AISValue3,Rebate4,RebateDefault4,MatrixValue4,AISValue4,Rebate5,RebateDefault5,MatrixValue5,AISValue5,Rebate6,RebateDefault6,MatrixValue6,AISValue6,Rebate7,RebateDefault7,MatrixValue7,AISValue7,Rebate8,RebateDefault8,MatrixValue8,AISValue8,Rebate9,RebateDefault9,MatrixValue9,AISValue9,Rebate10,RebateDefault10,MatrixValue10,AISValue10,Rebate11,RebateDefault11,MatrixValue11,AISValue11,Rebate12,RebateDefault12,MatrixValue12,AISValue12,Rebate13,RebateDefault13,MatrixValue13,AISValue13
0,2024 Mercedes AMG C43 4MATIC Sedan Sedan,2024,Mercedes Benz Financial,C43W4 C 43W4,C-Class,C43W4 4dr Sdn C43 AMG 4MATIC,3/31/2025 12:00:00 AM,20000.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,MBFS Star Loyalty Bonus Program,N,2000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,9000.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,MBFS Reignite Bonus Program,N,2000.0,0.0,Mercedes Incentive Bonus,Y,7000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024 Mercedes AMG C63 E Performance Sedan,2024,Mercedes Benz Financial,C63W4SE C 63W4SE,C-Class,C63W4SE 4dr Sdn C63 S E PERFORMANCE AMG 4MATIC,3/31/2025 12:00:00 AM,20000.0,0.0,Rising Star Program,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,2000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,9000.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,7000.0,0.0,MBFS Reignite Bonus Program,N,2000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df2.shape

(125, 61)

In [9]:
df = pd.merge(df1,df2,on='Bank')

In [10]:
df = df[['Package', 'Rate Model', 'Bank', 'MSRP', 'Invoice', 'Formula', 'Buying','MatrixPackage', 'Year', 'Code', 'MatrixRateModel', 'MatrixRateStyle',
       'Default Rebates', 'Nondefault Rebates', 'Rate Style', 'MF 12', 'RP 12',
       'Live 12', 'MF 24', 'RP 24', 'Live 24', 'MF 27', 'RP 27', 'Live 27',
       'MF 30', 'RP 30', 'Live 30', 'MF 33', 'RP 33', 'Live 33', 'MF 36',
       'RP 36', 'Live 36', 'MF 39', 'RP 39', 'Live 39', 'MF 42', 'RP 42',
       'Live 42', 'MF 48', 'RP 48', 'Live 48', 'Package Id', 'Bank Id',
       'EndDate', 'MatirxRebatesSum', 'AISRebatesSum', 'Rebate1',
       'RebateDefault1', 'MatrixValue1', 'AISValue1', 'Rebate2',
       'RebateDefault2', 'MatrixValue2', 'AISValue2', 'Rebate3',
       'RebateDefault3', 'MatrixValue3', 'AISValue3', 'Rebate4',
       'RebateDefault4', 'MatrixValue4', 'AISValue4', 'Rebate5',
       'RebateDefault5', 'MatrixValue5', 'AISValue5', 'Rebate6',
       'RebateDefault6', 'MatrixValue6', 'AISValue6', 'Rebate7',
       'RebateDefault7', 'MatrixValue7', 'AISValue7', 'Rebate8',
       'RebateDefault8', 'MatrixValue8', 'AISValue8', 'Rebate9',
       'RebateDefault9', 'MatrixValue9', 'AISValue9', 'Rebate10',
       'RebateDefault10', 'MatrixValue10', 'AISValue10', 'Rebate11',
       'RebateDefault11', 'MatrixValue11', 'AISValue11', 'Rebate12',
       'RebateDefault12', 'MatrixValue12', 'AISValue12', 'Rebate13',
       'RebateDefault13', 'MatrixValue13', 'AISValue13']]

In [11]:
result_df = df[df['Rate Style']==df['MatrixRateStyle']]

In [12]:
result_df['package_year']=result_df['Package'].apply(lambda x: int(x.split()[0]))

/var/folders/qd/53v0q6g93r5g_dc_bwl8_tlw0000gn/T/ipykernel_36404/767644778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['package_year']=result_df['Package'].apply(lambda x: int(x.split()[0]))


In [13]:
result_df['Make'] = "Mercedes"

/var/folders/qd/53v0q6g93r5g_dc_bwl8_tlw0000gn/T/ipykernel_36404/457980114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Make'] = "Mercedes"


In [14]:
result_df = result_df[result_df.Year==result_df.package_year]

In [15]:
result_df.shape

(1418, 101)

In [17]:
def car_details_extraction(car,style):
    try:

        response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": "You are a car expert. you will be given a car details, extract the following as json\n\nYear\nMake\nModel\nbody_type\ndrive_train\nStyle "
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": "2024 Acura ZDX AWD SUV \n 3B2H4REW 4dr Wgn A-Spec"
                }
              ]
            },
            {
              "role": "assistant",
              "content": [
                {
                  "type": "text",
                  "text": "{\n  \"Year\": 2024,\n  \"Make\": \"Acura\",\n  \"Model\": \"ZDX\",\n  \"drive_train\": \"AWD\"\n\"body_type\": \"SUV\"\n\"Style\": \"A-Spec\"\n}"
                }
              ]
            },
              {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": f'{car} \n {style}'
                }
              ]}

          ],
          response_format={
            "type": "json_object"
          },
          temperature=0,
          max_completion_tokens=2048,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        # print(response.choices[0].message.content)
        return response.choices[0].message.content
        
    except:
        print(car)
        return None

In [18]:
result_df["car_details"] = result_df.progress_apply(lambda x: car_details_extraction(x['Package'],x['Rate Style']),axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1418/1418 [42:57<00:00,  1.82s/it]


In [20]:
df = result_df.copy()

In [22]:

# df['Year'] = df['car_details'].apply(
#             lambda x: json.loads(x)["Year"] if isinstance(x, str) else pd.NA
#         )
# df['Make'] = df['car_details'].apply(
#             lambda x: json.loads(x)["Make"] if isinstance(x, str) else pd.NA
        # )
df['Model'] = df['car_details'].apply(
            lambda x: json.loads(x)["Model"] if isinstance(x, str) else pd.NA
        )
df['drive_train'] = df['car_details'].apply(
            lambda x: json.loads(x)["drive_train"] if isinstance(x, str) else pd.NA
        )
df['body_type'] = df['car_details'].apply(
            lambda x: json.loads(x)["body_type"] if isinstance(x, str) else pd.NA
        )
df['Style'] = df['car_details'].apply(
            lambda x: json.loads(x)["Style"] if isinstance(x, str) else pd.NA
        )

In [23]:
df.rename({""})

,Package,Rate Model,Bank,MSRP,Invoice,Formula,Buying,MatrixPackage,Year,Code,MatrixRateModel,MatrixRateStyle,Default Rebates,Nondefault Rebates,Rate Style,MF 12,RP 12,Live 12,MF 24,RP 24,Live 24,MF 27,RP 27,Live 27,MF 30,RP 30,Live 30,MF 33,RP 33,Live 33,MF 36,RP 36,Live 36,MF 39,RP 39,Live 39,MF 42,RP 42,Live 42,MF 48,RP 48,Live 48,Package Id,Bank Id,EndDate,MatirxRebatesSum,AISRebatesSum,Rebate1,RebateDefault1,MatrixValue1,AISValue1,Rebate2,RebateDefault2,MatrixValue2,AISValue2,Rebate3,RebateDefault3,MatrixValue3,AISValue3,Rebate4,RebateDefault4,MatrixValue4,AISValue4,Rebate5,RebateDefault5,MatrixValue5,AISValue5,Rebate6,RebateDefault6,MatrixValue6,AISValue6,Rebate7,RebateDefault7,MatrixValue7,AISValue7,Rebate8,RebateDefault8,MatrixValue8,AISValue8,Rebate9,RebateDefault9,MatrixValue9,AISValue9,Rebate10,RebateDefault10,MatrixValue10,AISValue10,Rebate11,RebateDefault11,MatrixValue11,AISValue11,Rebate12,RebateDefault12,MatrixValue12,AISValue12,Rebate13,RebateDefault13,MatrixValue13,AISValue13,package_year,Make,car_details,Model,drive_train,body_type,Style
2,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,110895.0,104798.0,0.945020,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64763,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,N,3000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,13000.0,0.0,NaN,NaN,NaN,NaN,2024,Mercedes,"{\n ""Year"": 2024,\n ""Make"": ""Mercedes"",\n ""...",AMG EQE,4MATIC+,SUV,EQE AMG
127,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,116005.0,109601.0,0.944795,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64764,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,N,3000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,13000.0,0.0,NaN,NaN,NaN,NaN,2024,Mercedes,"{\n ""Year"": 2024,\n ""Make"": ""Mercedes"",\n ""...",AMG EQE,4MATIC+,SUV,EQE AMG
252,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,117270.0,110790.0,0.944743,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64765,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,

In [24]:
df.to_csv("../data/benz_merged.csv",index=False)

In [32]:
df['Selling Price'] = df['Invoice']-df['Buying']

In [34]:
df['Adjusted Cap Cost'] = df['Selling Price']

In [35]:
acura_df = pd.read_csv("../data/merged_data_v4.csv")

In [36]:
final_df = pd.concat([acura_df,df])

In [39]:
final_df[acura_df.columns].to_csv("../data/merged_data_v5.csv")